In [1]:
import os

from classes.VAEGAN import CVAEGAN
from utils.callbacks import WandbImagesGAN, SaveGeneratorWeights, SaveVAEGANWeights, WandbImagesCVAEGAN
import tensorflow as tf
from tensorflow import keras
import numpy as np
import wandb
from wandb.keras import WandbCallback

In [2]:

wandb.login()

config={"dataset":"mnist","type":"C-VAE-GAN"}


## DATA
###
BS=256

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

ts=len(x_train)//BS

x_train=np.expand_dims(x_train.astype("float32")/255.,-1)
x_test=np.expand_dims(x_test.astype("float32")/255.,-1)

y_train=np.expand_dims(y_train.astype("int32"),-1)
y_test=np.expand_dims(y_test.astype("int32"),-1)


train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_dataset=  train_dataset.shuffle(buffer_size=1024).batch(BS).repeat()


test_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_dataset=test_dataset.shuffle(1024).batch(BS)

wandb: Currently logged in as: matteoferrante (use `wandb login --relogin` to force relogin)


In [3]:
encoder_architecture=[(0,64),(0,128)]
decoder_architecture=[(0,512),(0,384)]

discriminator_architecture=[(1,128),(0,256),(0,256)]
discriminator_dense=500

g=CVAEGAN((28,28,1),100,conditional_shape=(1,),n_classes=10,n_emb=50,encoder_architecture=encoder_architecture,decoder_architecture=decoder_architecture,discriminator_architecture=discriminator_architecture,discriminator_dense=discriminator_dense)



In [4]:
for x in train_dataset:
    break

g.vae(x)
g.vae.summary()

Model: "cvae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conditional_encoder (Functi  [(None, 100),            1370156   
 onal)                        (None, 100),                       
                              (None, 100)]                       
                                                                 
 conditional_decoder (Functi  (None, 28, 28, 1)        2393080   
 onal)                                                           
                                                                 
Total params: 3,763,242
Trainable params: 3,763,236
Non-trainable params: 6
_________________________________________________________________


In [5]:
g.discriminator.summary()

Model: "conditional_discriminator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 condition (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 condition_embeddings (Embeddin  (None, 1, 50)       50          ['condition[0][0]']              
 g)                                                                                               
                                                                                                  
 dense_3 (Dense)                (None, 1, 784)       39984       ['condition_embeddings[0][0]']   
                                                                                                  
 image_input (InputLayer)       [(None, 28, 28, 1)]  0           []       

In [6]:
wandb.init(project="TorVergataExperiment-Generative",config=config)


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [7]:

##CHECKPOINT
os.makedirs("models/cvaegan_mnist", exist_ok=True)
model_check=SaveVAEGANWeights(filepath="models/cvaegan_mnist")




callbacks=[
    WandbImagesCVAEGAN(test_dataset),
    WandbCallback(),
    model_check
]



In [8]:
### TRAINING

g.compile()
g.fit(train_dataset,steps_per_epoch=ts,epochs=40,callbacks=callbacks)



Epoch 1/40
234/234 [==============================] - 199s 836ms/step - d_acc: 0.0760 - d_loss: 0.3240 - vae_loss: 155.7081 - kl_loss: 38.0568
Epoch 2/40
234/234 [==============================] - 198s 842ms/step - d_acc: 0.0851 - d_loss: 0.2106 - vae_loss: 128.3436 - kl_loss: 37.7331
Epoch 3/40
234/234 [==============================] - 197s 843ms/step - d_acc: 0.0685 - d_loss: 0.1711 - vae_loss: 128.9633 - kl_loss: 39.8048
Epoch 4/40
234/234 [==============================] - 199s 851ms/step - d_acc: 0.0574 - d_loss: 0.2022 - vae_loss: 131.6832 - kl_loss: 38.2021
Epoch 5/40
234/234 [==============================] - 199s 850ms/step - d_acc: 0.0537 - d_loss: 0.2479 - vae_loss: 135.3800 - kl_loss: 43.1656
Epoch 6/40
234/234 [==============================] - 198s 845ms/step - d_acc: 0.0472 - d_loss: 0.1626 - vae_loss: 137.0521 - kl_loss: 40.0105
Epoch 7/40
234/234 [==============================] - 196s 840ms/step - d_acc: 0.0403 - d_loss: 0.1841 - vae_loss: 141.6291 - kl_loss: 35.8436

KeyboardInterrupt: 